### 데이터 불러오기

In [33]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
import keras.utils
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [23]:
# 새로운 데이터 (EDA)과정 진행 후 데이터
X_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/X_basic_train.csv')
X_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/X_basic_test.csv')
X_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/X_basic_val.csv')
y_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/y_basic_train.csv')
y_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/y_basic_test.csv')
y_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/Basic/y_basic_val.csv')

In [39]:
# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [24]:
y_train = y_train['class']
y_test = y_test['class']
y_val = y_val['class']

In [25]:
# 타겟 맵핑				
label_map = {'Pastry': 0, 'Z_Scratch': 1, 'K_Scatch': 2, 'Stains': 3, 'Dirtiness': 4, 'Bumps': 5, 'Other_Faults': 6}
y_train = [label_map[label] for label in y_train]
y_test = [label_map[label] for label in y_test]
y_val = [label_map[label] for label in y_val]

In [34]:
# y_train을 one-hot encoding으로 변환합니다
num_classes = len(set(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

### 모델 구현 
- 임시로 구현

In [40]:
# 인터넷 참고
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 모델구성
model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='relu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim//2, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer=custom_opt(0.005), metrics=['accuracy'])

early_stopping = EarlyStopping(patience=20, monitor='val_accuracy')

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)

print("validation loss, validation accuracy")
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

           

validation loss, validation accuracy
10/10 - 0s - loss: 0.7985 - accuracy: 0.7428 - 39ms/epoch - 4ms/step


In [42]:
# 복습한 내용으로 추가적인 진행
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.2269 - accuracy: 0.7331 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2295 - accuracy: 0.7621 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2116 - accuracy: 0.7428 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2683 - accuracy: 0.7460 - 37ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2861 - accuracy: 0.7138 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1867 - accuracy: 0.7524 - 28ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2240 - accuracy: 0.7428 - 46ms/epoch - 5ms/step
10/10 - 0s - loss: 0.2334 - accuracy: 0.7203 - 38ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2536 - accuracy: 0.7299 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2987 - accuracy: 0.7299 - 34ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2132 - accuracy: 0.7428 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2043 - accuracy: 0.7781 - 37ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2811 - accuracy: 0.5916 - 34ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2652 - accuracy: 0.7203 - 36ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2181 - accura

In [43]:
# 복습한 내용으로 추가적인 진행 - 다이아몬드형
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(512, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가   
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))   
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.2085 - accuracy: 0.7363 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2015 - accuracy: 0.7524 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2185 - accuracy: 0.7588 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2483 - accuracy: 0.7138 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2404 - accuracy: 0.7010 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1757 - accuracy: 0.7653 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2264 - accuracy: 0.7653 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2238 - accuracy: 0.7170 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2166 - accuracy: 0.7492 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2036 - accuracy: 0.6817 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1743 - accuracy: 0.7685 - 36ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2292 - accuracy: 0.7331 - 43ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2007 - accuracy: 0.7524 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2317 - accuracy: 0.7460 - 35ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2470 - accura

In [44]:
# 복습한 내용으로 추가적인 진행 - 다이아몬드형
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(512, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(256, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가   
            model.add(Dense(96, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(64, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(16, activation=func))
            model.add(Dense(12, activation=func)) 
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.1935 - accuracy: 0.7460 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2113 - accuracy: 0.7363 - 34ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2243 - accuracy: 0.7428 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2505 - accuracy: 0.7556 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2912 - accuracy: 0.7106 - 41ms/epoch - 4ms/step
10/10 - 0s - loss: 0.1828 - accuracy: 0.7460 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2031 - accuracy: 0.7460 - 38ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2442 - accuracy: 0.7395 - 38ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2647 - accuracy: 0.7363 - 34ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2666 - accuracy: 0.7203 - 36ms/epoch - 4ms/step
10/10 - 0s - loss: 0.1852 - accuracy: 0.7428 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1976 - accuracy: 0.7395 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2175 - accuracy: 0.7331 - 35ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2544 - accuracy: 0.7653 - 38ms/epoch - 4ms/step
10/10 - 0s - loss: 0.2620 - accura